# SDialog dependencies

In [ ]:
# Setup the environment depending on weather we are running in Google Colab or Jupyter Notebook
import os
from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    print("Running on CoLab")

    # Installing Ollama (if you are not planning to use Ollama, you can just comment these lines to speed up the installation)
    !curl -fsSL https://ollama.com/install.sh | sh

    # Installing sdialog
    !git clone https://github.com/qanastek/sdialog.git
    %cd sdialog
    %pip install -e .
    %cd ..
else:
    print("Running in Jupyter Notebook")
    # Little hack to avoid the "OSError: Background processes not supported." error in Jupyter notebooks"
    get_ipython().system = os.system

## Locally

Create a `.venv` using the root `requirement.txt` file and Python `3.11.14`

# Tutorial 10: Audio generation with room accoustic

In order to run the next steps in a fast manner, we will start from an existing dialog generated using previous tutorials. If you haven't download yet `demo_dialog_doctor_patient.json` locally, please download it from our GitHub repository using the following command:

In [ ]:
# If demo_dialog_doctor_patient.json is not present, download it
if os.path.exists("demo_dialog_doctor_patient.json"):
    print("demo_dialog_doctor_patient.json already exists")
else:
    !wget https://raw.githubusercontent.com/qanastek/sdialog/refs/heads/main/tests/data/demo_dialog_doctor_patient.json

In [ ]:
from sdialog import Dialog

# Load dialog

In [ ]:
original_dialog = Dialog.from_file("demo_dialog_doctor_patient.json")
original_dialog.print()

# Audio Generation

### Instanciate voices database

In [ ]:
from sdialog.audio.voice_database import HuggingfaceVoiceDatabase
dummy_voice_database = HuggingfaceVoiceDatabase("sdialog/voices-kokoro")
dummy_voice_database.get_voice(gender="male", age=20)

### Instanciate TTS model

In [ ]:
!pip install -q kokoro>=0.9.4
!apt-get -qq -y install espeak-ng > /dev/null 2>&1

In [ ]:
from sdialog.audio.tts_engine import KokoroTTS
tts_engine = KokoroTTS()

## Setup stage: Audio Dialog and Audio Pipeline

In [ ]:
from sdialog.audio.audio_dialog import AudioDialog
from sdialog.audio.audio_pipeline import AudioPipeline

Convert the original dialog into a audio enhanced dialog

In [ ]:
dialog: AudioDialog = AudioDialog.from_dialog(original_dialog)

Instanciate the audio pipeline in order to use `Kokoro` (`tts_engine`) as the TTS model and save the audios outputs of all the dialogs into the directory `./audio_outputs`.

The voices are sampled from the `dummy_voice_database` based on the persona attributes `age` and `gender`, as assigned during the original textual dialog.

In [ ]:
!git clone https://github.com/cyrta/dscaper.git
!pip install -e ./dscaper/
!pip install scaper

If you get `ModuleNotFoundError: No module named 'scaper'` just `Restart` your environment.

In [ ]:
import scaper
DATA_PATH = "./dscaper_data" # Path where the sound events, utterances and timelines database will be saved
os.makedirs(DATA_PATH, exist_ok=True)

In [ ]:
dsc = scaper.Dscaper(dscaper_base_path=DATA_PATH)

In [ ]:
os.makedirs("./audio_outputs", exist_ok=True)
audio_pipeline = AudioPipeline(
    voice_database=dummy_voice_database,
    tts_pipeline=tts_engine,
    dscaper=dsc,
    dir_audio="./audio_outputs",
)
# audio_pipeline = AudioPipeline() # Can also be used with default values

In [ ]:
# Populate the sound events database
audio_pipeline.populate_dscaper(["sdialog/background","sdialog/foreground"])

or if you encounter any issue during the download due to timeout:

In [ ]:
%%script false --no-raise-error
!hf download sdialog/background --repo-type dataset
!hf download sdialog/foreground --repo-type dataset

Now let's generate a medical room it will be enough and display it's shape and content:

In [ ]:
from sdialog.audio.audio_utils import SourceVolume, SourceType
from sdialog.audio.jsalt import MedicalRoomGenerator, RoomRole

In [ ]:
room = MedicalRoomGenerator().generate(args={"room_type": RoomRole.EXAMINATION})
img = room.to_image()
display(img)
img.save("room.png")

We can also optionaly position the speakers 1 and 2 into the room by assigning them places around available furnitures in the room or to a 3D position in the environment.

By default the speakers are positionned around the center of the room.

Here the `MedicalRoomGenerator` is generating rooms with a predefined list of furnitures (desk, sink, door, ...) that can be used:

In [ ]:
from sdialog.audio.room import SpeakerSide, Role, RoomPosition

In [ ]:
room.place_speaker_around_furniture(speaker_name=Role.SPEAKER_1, furniture_name="desk", max_distance=1.0, side=SpeakerSide.FRONT)
room.place_speaker_around_furniture(speaker_name=Role.SPEAKER_2, furniture_name="desk", max_distance=1.0, side=SpeakerSide.BACK)

You can see the new positions of the speakers:

In [ ]:
img = room.to_image()
display(img)

Perform the inference of the audio pipeline on the previously converted dialog. In this case we will focus on generating the "unprocessed" audio, which consist of the agregation of all utterances from the dialog. Rather than using the dialog identifier as the name of the directory, we are using here a custom directory name `demo_dialog_room_accoustic` which will be saved at `./audio_outputs/demo_dialog_room_accoustic/`. 

In [ ]:
dialog: AudioDialog = audio_pipeline.inference(
    dialog,
    environment={
        "room": room, # Need to provide a room object to trigger the 3rd step of the audio pipeline
        "background_effect": "white_noise",
        "foreground_effect": "ac_noise_minimal",
        "foreround_effect_position": RoomPosition.TOP_RIGHT,
        "source_volumes": {
            SourceType.ROOM: SourceVolume.HIGH,
            SourceType.BACKGROUND: SourceVolume.VERY_LOW
        },
        "kwargs_pyroom": {
            "ray_tracing": True,
            "air_absorption": True
        }
    },
    do_step_1=True,
    do_step_2=True,
    do_step_3=True,
    dialog_dir_name="demo_dialog_room_accoustic",
    room_name="my_room_config_1"
)
print(dialog.audio_step_1_filepath)
print(dialog.audio_step_2_filepath)
print(dialog.audio_step_3_filepaths)

In [ ]:
from IPython.display import Audio, display

display(Audio(dialog.audio_step_1_filepath, autoplay=False))
display(Audio(dialog.audio_step_2_filepath, autoplay=False))
print("-"*25)
print("- Room Configurations")
print("-"*25)
for config_name in dialog.audio_step_3_filepaths:
    print(f"> Room Configuration: {config_name}")
    display(Audio(dialog.audio_step_3_filepaths[config_name]["audio_path"], autoplay=False))